In [ ]:
import csv
import pandas as pd
import clip
import torch
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from PIL import Image
from scipy.spatial.distance import cdist
import bisect 
from scipy.spatial import ConvexHull
import gc
import debias_clip as dclip
import copy
import sklearn.feature_selection as fs
from image_database import *
#import cvxpy as cp
image_folder_prefix = 'datasets/gender_in_image_search/google/'

data = pd.read_csv('datasets/gender_in_image_search/gender_labelled_images.csv')

In [ ]:
## Preprocessing

catagories = data['search_term'].unique()
data['image_relative_path'] = data['image_url'].str.replace("http://homes.cs.washington.edu/~mjskay/aws-images/google/", image_folder_prefix)

print(len(catagories))
print(data.shape)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device)

batch_size = 200

features = process_images(model, preprocess, data['image_relative_path'], batch_size=batch_size)



In [ ]:
device_d = "cuda" if torch.cuda.is_available() else "cpu"
model_debias, preprocess_debias = dclip.load("ViT-B/16-gender", device_d)

batch_size = 200

features_debias = process_images(model_debias, preprocess_debias, data['image_relative_path'], batch_size=batch_size, device=device_d)

device_d = 'cpu'
model_debias, preprocess_debias = dclip.load("ViT-B/16-gender", device_d)
    



In [ ]:
image_database = ImageDatabase(features, data, model, preprocess, device)
image_database.sensitive_attributes([("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person"), ("A picture of a young person", "A picture of an old person")])

#print(image_database.search("This is a picture of a engineer."))
print(image_database.distinct_retrival("This is a picture of a pilot.", method='feature_distances', k=10))

In [ ]:
image_database = ImageDatabase(features, data, model, preprocess, device)
image_database.sensitive_attributes([("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person")]) #
image_database.define_pbm_classes(["unknown gender", "man", "woman"])
debias_database = ImageDatabase(features_debias, data, model_debias, preprocess_debias, device_d)

baseline_metrics = {'name': 'Baseline'}
random_from_similar_set_metrics = {'name': 'RandomSS'}
divimage_max_sum_metrics = {'name': 'DivImageMSum'}
divimage_max_min_metrics = {'name': 'DivImageMMin'}
#diviamge_clustering_metrics = {'name': 'DivImageC'}
#divimage_qp_metrics = {'name': 'DivImageQP'}
random_from_similar_set_metrics_larger_tol = {'name': 'RandomSS'}
divimage_max_sum_metrics_larger_tol = {'name': 'DivImageMSum'}
divimage_max_min_metrics_larger_tol = {'name': 'DivImageMMin'}
pbm_metrics = {'name': 'PBM'}
pbm_intersectional_metrics = {'name': 'PBM_Intersectional'}
debias_clip_metrics = {'name': 'DebiasClip'}
true_labels = {'name': 'TrueConceptLabels'}
true_labels_larger_tol = {'name': 'TrueConceptLabels'}


indistinguisable_values = [[cat] for cat in catagories]
true_rates = [[data[data['search_term'] == cat].iloc[0].search_p_women for cat in catagories]]
totals_by_cat = {cat: len(data[data['search_term'] == cat]) for cat in catagories}
image_database.define_coordinate_mapping(['image_gender'], [['man']], [['woman']])

run_analysis(lambda x: image_database.search(x, 50), 50, None, baseline_metrics, catagories, 'search_term', indistinguisable_values, ['image_gender'], [['woman']], [['man']], true_rates, totals_by_cat)
run_analysis(lambda x: image_database.distinct_retrival(x, 50, tol=.03, method='max_sum'), 50, .03, divimage_max_sum_metrics, catagories, 'search_term', indistinguisable_values, ['image_gender'], [['woman']], [['man']], true_rates, totals_by_cat)

In [ ]:
print(baseline_metrics.keys())
#print(baseline_metrics['bias'])
divimage_max_sum_metrics

plt.scatter(baseline_metrics['bias'], baseline_metrics['precision'], alpha=.8)
plt.scatter(divimage_max_sum_metrics['bias'], divimage_max_sum_metrics['precision'], alpha=.8)
plt.title("Bias vs Precision on GIIS", weight='bold')
plt.xlabel("Gender Bias", weight='bold', fontsize=12)
plt.ylabel("Precision", weight='bold', fontsize=12)
plt.legend(["Baseline", "CDI-MaxSum, tol = .03"], loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=2)
plt.show()


In [ ]:

image_database = ImageDatabase(features, data, model, preprocess, device)
totals_by_cat = {cat: len(data[data['search_term'] == cat]) for cat in catagories}
image_database.define_coordinate_mapping(['image_gender'], [['man']], [['woman']])
indistinguisable_values = [[cat] for cat in catagories]
true_rates = [[data[data['search_term'] == cat].iloc[0].search_p_women for cat in catagories]]
totals_by_cat = {cat: len(data[data['search_term'] == cat]) for cat in catagories}

image_database.add_clipclip_ordering("gender", np.load('datasets/MI_orders/gender.npy'))
image_database.add_clipclip_ordering("skintone", np.load('datasets/MI_orders/skintone.npy'))
image_database.add_clipclip_ordering("intersectional", np.load('datasets/MI_orders/gender_skintone.npy'))
image_database.add_clipclip_ordering("three_attr", np.load('datasets/MI_orders/intersectional.npy'))


"""method_name_specification_list = [
    (lambda k, tol: lambda x: image_database.search(x, k), 'Baseline', []),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_sum'), 'CDI_Sum_gender', [("sensitive_attributes", [("A picture of a man", "A picture of a woman")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_sum'), 'CDI_Sum_intersectional', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_sum'), 'CDI_Sum_skintone', [("sensitive_attributes", [("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_gender', [("sensitive_attributes", [("A picture of a man", "A picture of a woman")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_intersectional', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_skintone', [("sensitive_attributes", [("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='random'), 'CDI_Random', [("sensitive_attributes", [("A picture of a man", "A picture of a woman")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='true_labels'), 'CDI_TrueConcept', []),
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_gender', [("pbm_classes", ["unknown gender", "man", "woman"])]),
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_intersectional', [("pbm_classes", ["unknown gender and skin-tone", "light-skinned man", "light-skinned woman", "dark-skinned man", "dark-skinned woman"])]),
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_skintone', [("pbm_classes", ["unknown skin-tone", "light-skinned person", "dark-skinned person"])]),
    (lambda k, tol: lambda x: debias_database.search(x, k), 'DebiasClip', [])
]"""

"""method_name_specification_list = [
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_gender', [("pbm_classes", ["unknown gender", "man", "woman"])]),
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_intersectional', [("pbm_classes", ["unknown gender and skin-tone", "light-skinned man", "light-skinned woman", "dark-skinned man", "dark-skinned woman"])]),
    (lambda k, eps: lambda x: image_database.pbm(x, k, eps=eps), 'PBM_skintone', [("pbm_classes", ["unknown skin-tone", "light-skinned person", "dark-skinned person"])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='feature_distances'), 'CDI_Features', [])
]"""

method_name_specification_list = [
    (lambda k, n: lambda x: image_database.clip_clip(x, "gender", n, k), 'CLIP_gender', []),
    (lambda k, n: lambda x: image_database.clip_clip(x, "skintone", n, k), 'CLIP_skintone', []),
    (lambda k, n: lambda x: image_database.clip_clip(x, "intersectional", n, k), 'CLIP_intersectional', []),
    (lambda k, n: lambda x: image_database.clip_clip(x, "three_attr", n, k), 'CLIP_three_attributes', []),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='max_min'), 'CDI_Min_skintone', [("sensitive_attributes", [("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='euc_max_sum'), 'CDI_EucSum_intersectional', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person")])]),
    (lambda k, tol: lambda x: image_database.distinct_retrival(x, k, tol=tol, method='euc_max_min'), 'CDI_EucMin_intersectional', [("sensitive_attributes", [("A picture of a man", "A picture of a woman"), ("A picture of a light-skinned person", "A picture of a dark-skinned person")])])
]

ks = [10, 25, 50, 100]

number_of_tol_steps = 16
number_of_eps_steps = 11
random_iters = 4
number_of_clip_clip_steps = 24

result_dicts = []

for method, name, spec in method_name_specification_list:
    print(f"Starting analysis for method: {name}...")
    for s, val in spec:
        if s == "sensitive_attributes":
            image_database.sensitive_attributes(val)
        if s == "pbm_classes":
            image_database.define_pbm_classes(val)
    for k in tqdm(ks):
        result_dict = {'name': name}

        if name in ['Baseline', "DebiasClip"]:
            steps = 1
        else:
            steps = number_of_tol_steps

        if name[0:3] == 'PBM':
            for e in reversed(range(0, number_of_eps_steps)):
                eps = e / (number_of_eps_steps - 1)
                retrieval_function = method(k, eps)
                new_dict = result_dict.copy()
                random_results = []
                for i in range(random_iters):
                    new_dict = result_dict.copy()
                    run_analysis(retrieval_function, k, eps, new_dict, catagories, 'search_term', indistinguisable_values,  ['image_gender'], [['woman']], [['man']], true_rates, totals_by_cat)
                    random_results.append(new_dict)
                
                add_dict = result_dict.copy()
                for key in random_results[0].keys():
                    if key == 'name':
                        continue
                    add_dict[key] = np.mean([res[key] for res in random_results], axis=0)
                result_dicts.append(add_dict)
        
        elif name[0:4] == 'CLIP':
            for e in range(0, number_of_clip_clip_steps):
                n = e * 20
                retrieval_function = method(k, n)
                new_dict = result_dict.copy()
                run_analysis(retrieval_function, k, n, new_dict, catagories, 'search_term', indistinguisable_values, ['image_gender'],  [['woman']], [['man']], true_rates, totals_by_cat)
                result_dicts.append(new_dict)

        else:
            for t in range(0, steps):
                if steps == 1:
                    tol = None
                else:
                    tol = t / 200
                retrieval_function = method(k, tol)
                new_dict = result_dict.copy()
                if name == "CDI_Random":
                    random_results = []
                    for i in range(random_iters):
                        new_dict = result_dict.copy()
                        run_analysis(retrieval_function, k, tol, new_dict, catagories, 'search_term', indistinguisable_values,  ['image_gender'], [['woman']], [['man']], true_rates, totals_by_cat)
                        random_results.append(new_dict)
                    new_dict = result_dict.copy()

                    for key in random_results[0].keys():
                        if key == 'name':
                            continue
                        new_dict[key] = np.mean([res[key] for res in random_results], axis=0)
                else:
                    run_analysis(retrieval_function, k, tol, new_dict, catagories, 'search_term', indistinguisable_values, ['image_gender'], [['woman']], [['man']], true_rates, totals_by_cat)
                result_dicts.append(new_dict)

df = parse_analysis(result_dicts, ['gender'])

print(df)

df.to_pickle("results/giis-3-24-CLIP-euc.pkl")
        

In [ ]:
df2 = pd.read_pickle("results/giis-3-21.pkl")

print(df2)

In [ ]:
method_names = ['Baseline', 'DebiasClip', "CDI_Sum_gender"]
method_names = df2['name'].unique()

for k in [10, 25, 50, 100]:
    plot_across_tol(df2, 25, method_names, 'Avg_AbsBias_gender', 'Avg_Precision', reverse_x=True)
    plot_across_tol(df2, 25, method_names, 'Avg_Bias_gender', 'Avg_Precision', reverse_x=False)
    plot_across_tol(df2, 25, method_names, 'Avg_Skew_gender', 'Avg_Precision', reverse_x=False)
    plot_across_tol(df2, 25, method_names, 'Avg_AbsBias_gender', 'Avg_Recall', reverse_x=True)
    plot_across_tol(df2, 25, method_names, 'Avg_Bias_gender', 'Avg_Recall', reverse_x=False)
    plot_across_tol(df2, 25, method_names, 'Avg_AbsBias_for_Accurate_gender', 'Avg_Precision', reverse_x=True)